In [62]:
import pandas as pd
import xlrd
import pickle #modulo usado para serializar dados para o arquivo binario
import os
import time

df = pd.read_excel('Hot Stuff.xlsx')

In [63]:
#esta celula calcula o quao popular foi uma musica no mes especificado (Maio de 1990) atribuindo um sistema de pontos baseado em que posicao ela ficou no top 100 em cada semana do mês
#vai contar o quao popular foi a musica no ano atribuindo pontos a quantas semanas ela ficou entre as mais tocadas
contador = {}

#testando se o char especificado esta na entrada atual
#pode ser usado para identificar anos diferentes
ano = '2010'
meses = ['01','02','03','04','05','06','07','08','09','10','11','12']
tempo = time.process_time() #Armazena o tempo de inicio do processamento
for ano in range(1960,2016):
    for mes in meses:
        contador = {}
        for i in range(len(df)):
            if (str(ano) + '-' + mes + '-') in df['WeekID'][i]: #filtra todas as entradas por ano E mes nesse caso (Maio de 1990)
                if(df['SongID'][i] in contador): #testa se a musica ja esta no dicionario (ja foi computada)
                    contador[df['SongID'][i]]['Pontos'] += (101 - df['Week Position'][i])
                else: #cai aqui se ainda nao foi computada
                    contador[df['SongID'][i]] = {}
                    contador[df['SongID'][i]]['Artista'] = df['Performer'][i]
                    contador[df['SongID'][i]]['Titulo'] = df['Song'][i]
                    contador[df['SongID'][i]]['Ano'] = ano
                    contador[df['SongID'][i]]['Mes'] = mes
                    contador[df['SongID'][i]]['Pontos'] = (101 - df['Week Position'][i])
                    #A medida 'Peak' nao eh a posicao exata que ela atingiu mas os "pontos maximos" ganhos em uma semana (de 1 a 100)
                    contador[df['SongID'][i]]['Peak'] = (101 - df['Peak Position'][i]) #salva qual foi a maior posicao alcançada pela musica (usada para criterios de desempate)
                    contador[df['SongID'][i]]['Semanas'] = df['Weeks on Chart'][i] #Terceiro criterio de desempate
                
        #Faz o sort do mes atual
        items = sorted(contador.items(), key = lambda tup: (tup[1]["Pontos"], tup[1]["Peak"], tup[1]["Semanas"]), reverse=True)
    
        #armazena as primeiras 100 entradas em um dicionário mais organizado
        top = [dict() for x in range(100)] #cria uma lista de 100 dicionarios
        for i in range(len(top)):
            for y in items[i][1]:
                top[i][y] = items[i][1][y]
                
        #Concatena o dicionario no arquivo teste.bin
        filename = 'teste.bin'
        topBytes = []
    
        if os.path.exists(filename): #confere se o arquivo existe
            with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
                topBytes = pickle.load(openfile)


        newData = top #Aqui vem a nova data que deve ser concatenada
        
        topBytes = topBytes + newData #concatena o que ja tinha no arquivo binario com a nova data
        with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
            pickle.dump(topBytes, openfile) 
        
        #Cria o arquivo de indice para consultas    
        filename = 'indices.bin'
        fileBytes = []

        if os.path.exists(filename): #confere se o arquivo existe
            with open(filename,'rb') as openfile:  #with automaticamente da um close() no final
                fileBytes = pickle.load(openfile)


        newData = {'Ano': ano, 'Mes': mes, 'Min': len(topBytes) - 100, 'Max': len(topBytes) - 1} #Aqui vem a nova data que deve ser concatenada
        
        fileBytes.append(newData)#concatena o que ja tinha no arquivo binario com a nova data
        with (open(filename, 'wb+')) as openfile: #abre o arquivo de teste no modo de leitura binaria 'wb' como openfile
            pickle.dump(fileBytes, openfile) 
            
t = time.process_time() - tempo #Salva em t o tempo que o processo levou

KeyboardInterrupt: 

In [64]:
#Faz a leitura do arquivo de registros
with (open('teste.bin', 'rb')) as openfile:
    leitura = pickle.load(openfile)
    
with (open('indices.bin', 'rb')) as openfile:
    indices = pickle.load(openfile)

In [65]:
for i in indices:
    print(i)

{'Ano': 1960, 'Mes': '01', 'Min': 0, 'Max': 99}
{'Ano': 1960, 'Mes': '02', 'Min': 100, 'Max': 199}
{'Ano': 1960, 'Mes': '03', 'Min': 200, 'Max': 299}
{'Ano': 1960, 'Mes': '04', 'Min': 300, 'Max': 399}
{'Ano': 1960, 'Mes': '05', 'Min': 400, 'Max': 499}
{'Ano': 1960, 'Mes': '06', 'Min': 500, 'Max': 599}
{'Ano': 1960, 'Mes': '07', 'Min': 600, 'Max': 699}
{'Ano': 1960, 'Mes': '08', 'Min': 700, 'Max': 799}
{'Ano': 1960, 'Mes': '09', 'Min': 800, 'Max': 899}
{'Ano': 1960, 'Mes': '10', 'Min': 900, 'Max': 999}
{'Ano': 1960, 'Mes': '11', 'Min': 1000, 'Max': 1099}
{'Ano': 1960, 'Mes': '12', 'Min': 1100, 'Max': 1199}
{'Ano': 1961, 'Mes': '01', 'Min': 1200, 'Max': 1299}
{'Ano': 1961, 'Mes': '02', 'Min': 1300, 'Max': 1399}
{'Ano': 1961, 'Mes': '03', 'Min': 1400, 'Max': 1499}
{'Ano': 1961, 'Mes': '04', 'Min': 1500, 'Max': 1599}
{'Ano': 1961, 'Mes': '05', 'Min': 1600, 'Max': 1699}
{'Ano': 1961, 'Mes': '06', 'Min': 1700, 'Max': 1799}
{'Ano': 1961, 'Mes': '07', 'Min': 1800, 'Max': 1899}
{'Ano': 1961, 'M

In [18]:
print(len(leitura))

5500


### Teste da relação
consultando o indice inicial de um dado ano no arquivo indice e imprimindo os 5 mais populares daquele ano

In [19]:
start = 0

for x in indices:
    if x['Ano'] == 2014:
        start = x['Min']
 
for i in range(5):
    print(leitura[start + i])

{'Artista': 'John Legend', 'Titulo': 'All Of Me', 'Ano': 2014, 'Pontos': 4069, 'Peak': 100, 'Semanas': 59}
{'Artista': 'Pharrell Williams', 'Titulo': 'Happy', 'Ano': 2014, 'Pontos': 3898, 'Peak': 100, 'Semanas': 47}
{'Artista': 'Katy Perry Featuring Juicy J', 'Titulo': 'Dark Horse', 'Ano': 2014, 'Pontos': 3816, 'Peak': 100, 'Semanas': 57}
{'Artista': 'OneRepublic', 'Titulo': 'Counting Stars', 'Ano': 2014, 'Pontos': 3373, 'Peak': 99, 'Semanas': 68}
{'Artista': 'Iggy Azalea Featuring Charli XCX', 'Titulo': 'Fancy', 'Ano': 2014, 'Pontos': 2993, 'Peak': 100, 'Semanas': 39}


In [66]:
print(tempo)

1233.615231867
